In [2]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
# to use this you will have to install pandasql to your environment
# use: pip install pandasql
from pandasql import sqldf as sq
# call the lambda function pysqdf to run a querry on one of the tables using SQL, there are examples in the code further down
pysqdf = lambda q: sq(q, globals())

In [3]:
pkl_names = [    "genomeScores",
        "genomeTags",
        "links",
        "movies",
        "tags",
        "ratings"]


genomeScores = pd.read_pickle('pickle/genomeScores.pkl')
genomeTags = pd.read_pickle('pickle/genomeTags.pkl')
links = pd.read_pickle('pickle/links.pkl')
movies = pd.read_pickle('pickle/movies.pkl')
tags = pd.read_pickle('pickle/tags.pkl')
#you will need to make the ratings.pkl file, watch the tutorial that Ryan Put in the discord on how to make that. 
# try - except statement, if the part in the try errors out it will run the code in except, if the try code succeeds it will not run the except code. Google search for more information
try:
    ratings = pd.read_pickle('pickle/ratings.pkl')
    # creates a list of all the pkl files
    all_pkls = [    genomeScores,
        genomeTags,
        links,
        movies,
        tags,
        ratings]
except: 
    print("there is no ratings.pkl file in the pickle folder")
    try:
        ratings_csv = pd.read_csv('ml-25m/ratings.csv')
        # if there is no pkl file for raings add it as a csv file
        all_pkls = [    genomeScores,
            genomeTags,
            links,
            movies,
            tags,
            ratings_csv]
    except:
        print("raings.csv not found in the folder ml-25")
        # if there is no csv or pkl file for ratings do not include it in the list
        all_pkls = [    genomeScores,
            genomeTags,
            links,
            movies,
            tags]


In [17]:
print(genomeScores)

          movieId  tagId  relevance
0               1      1    0.02875
1               1      2    0.02375
2               1      3    0.06250
3               1      4    0.07575
4               1      5    0.14075
...           ...    ...        ...
15584443   206499   1124    0.11000
15584444   206499   1125    0.04850
15584445   206499   1126    0.01325
15584446   206499   1127    0.14025
15584447   206499   1128    0.03350

[15584448 rows x 3 columns]


In [18]:
print(GenomeTags)

      tagId           tag
0         1           007
1         2  007 (series)
2         3  18th century
3         4         1920s
4         5         1930s
...     ...           ...
1123   1124       writing
1124   1125         wuxia
1125   1126          wwii
1126   1127        zombie
1127   1128       zombies

[1128 rows x 2 columns]


In [19]:
print( links)

       movieId   imdbId    tmdbId
0            1   114709     862.0
1            2   113497    8844.0
2            3   113228   15602.0
3            4   114885   31357.0
4            5   113041   11862.0
...        ...      ...       ...
62418   209157  6671244  499546.0
62419   209159   297986   63407.0
62420   209163  6755366  553036.0
62421   209169   249603  162892.0
62422   209171    55323   79513.0

[62423 rows x 3 columns]


In [20]:
print(movies)

       movieId                               title  \
0            1                    Toy Story (1995)   
1            2                      Jumanji (1995)   
2            3             Grumpier Old Men (1995)   
3            4            Waiting to Exhale (1995)   
4            5  Father of the Bride Part II (1995)   
...        ...                                 ...   
62418   209157                           We (2018)   
62419   209159           Window of the Soul (2001)   
62420   209163                    Bad Poems (2018)   
62421   209169                 A Girl Thing (2001)   
62422   209171      Women of Devil's Island (1962)   

                                            genres  
0      Adventure|Animation|Children|Comedy|Fantasy  
1                       Adventure|Children|Fantasy  
2                                   Comedy|Romance  
3                             Comedy|Drama|Romance  
4                                           Comedy  
...                              

In [27]:
try: 
    print("ratings as a pkl: ", ratings)
except: 
    try:
        print("ratings as a csv: ", ratings_csv)
    except: 
        print("ratings does not excist as a csv or a pkl, please add it")


ratings as a pkl:            userId  movieId  rating   timestamp
0              1      296     5.0  1147880044
1              1      306     3.5  1147868817
2              1      307     5.0  1147868828
3              1      665     5.0  1147878820
4              1      899     3.5  1147868510
...          ...      ...     ...         ...
25000090  162541    50872     4.5  1240953372
25000091  162541    55768     2.5  1240951998
25000092  162541    56176     2.0  1240950697
25000093  162541    58559     4.0  1240953434
25000094  162541    63876     5.0  1240952515

[25000095 rows x 4 columns]


In [22]:
print(tags)

         userId  movieId                  tag   timestamp
0             3      260              classic  1439472355
1             3      260               sci-fi  1439472256
2             4     1732          dark comedy  1573943598
3             4     1732       great dialogue  1573943604
4             4     7569     so bad it's good  1573943455
...         ...      ...                  ...         ...
1093355  162521    66934  Neil Patrick Harris  1427311611
1093356  162521   103341     cornetto trilogy  1427311259
1093357  162534   189169               comedy  1527518175
1093358  162534   189169             disabled  1527518181
1093359  162534   189169              robbery  1527518193

[1093360 rows x 4 columns]


## Print the number of null values in every pkl file

In [3]:
for iter in range(len(all_pkls)):
        print(f"The table {pkl_names[iter]} has {len(all_pkls[iter][all_pkls[iter].isnull().any(axis=1)])} null values")


The table genomeScores has 0 null values
The table genomeTags has 0 null values
The table links has 107 null values
The table movies has 0 null values
The table tags has 16 null values
The table ratings has 0 null values


## Check for outliers in dataset

In [4]:
# q is a SQL query, format it like below and then you call pysqdf(q) and pass in your query "q" to run your query on the dataframes you specify
# in general for SQL, words that are all capitalized are build into the SQL language, things that have lowercase letters are parts of you database/dataframe

# Checking genomeScore for outliers on min and max
# Check the top 
q = """SELECT *
    FROM genomeScores
    WHERE relevance <> 1
    ORDER BY relevance DESC
    LIMIT 10;"""
q2 = """SELECT *
    FROM genomeScores
    WHERE relevance <> 1
    ORDER BY relevance
    LIMIT 10;"""
print(pysqdf(q))
print(pysqdf(q2))


   movieId  tagId  relevance
0       44   1075    0.99975
1      150    944    0.99975
2      356    752    0.99975
3      903    511    0.99975
4      904    511    0.99975
5      965    511    0.99975
6     1213    741    0.99975
7     1231    944    0.99975
8     1241    956    0.99975
9     1245    224    0.99975


KeyboardInterrupt: 

In [28]:
# Checking genomeTags for outliers on min and max
# Check the top 
q = """SELECT *
    FROM genomeTags
    ORDER BY tag DESC
    LIMIT 10;"""
# Check the bottom 10
q2 = """SELECT *
    FROM genomeTags
    ORDER BY tag
    LIMIT 10;"""
print(pysqdf(q))
print(pysqdf(q2))

   tagId             tag
0   1128         zombies
1   1127          zombie
2   1126            wwii
3   1125           wuxia
4   1124         writing
5   1123         writers
6   1122   writer's life
7   1121    world war ii
8   1120     world war i
9   1119  world politics
   tagId           tag
0      1           007
1      2  007 (series)
2      3  18th century
3      4         1920s
4      5         1930s
5      6         1950s
6      7         1960s
7      8         1970s
8      9         1980s
9     10  19th century


In [30]:
# Checking tags for outliers on min and max
# Check the top 10 timestamps
q = """SELECT *
    FROM tags
    ORDER BY timestamp DESC
    LIMIT 10;"""
# Check the bottom 10 timestamps
q2 = """SELECT *
    FROM tags
    ORDER BY timestamp
    LIMIT 10;"""
print(pysqdf(q))
print(pysqdf(q2))

   userId  movieId           tag   timestamp
0  123527   201594       villain  1574316696
1  123527   201594  country life  1574316682
2  123527   201594     farmhouse  1574316680
3  123527   201594          farm  1574316678
4  123527   201594        kansas  1574316676
5  123527   201594      thriller  1574316667
6  123527   201594        sci-fi  1574316666
7  123527   201594        horror  1574316664
8  123527   201594          gore  1574316613
9  123527   201594  supernatural  1574316603
   userId  movieId              tag   timestamp
0  129396     2788     monty python  1135429210
1  129396     1732    coen brothers  1135429236
2  129396     1206  stanley kubrick  1135429248
3  129396     1193   jack nicholson  1135429371
4  129396     5004    peter sellers  1135429399
5  129396       47        brad pitt  1135429412
6  129396       47   morgan freeman  1135429412
7  129396     4011        brad pitt  1135429431
8  129396     4011      guy ritchie  1135429431
9  129396       32     br

In [9]:
# Checking ratings for outliers on min and max
# Check the top 10 timestamps
q = """SELECT userId, avg(rating) as avg_rating, count(movieId) as review_count
    FROM ratings
    group by userId;"""
# Check the bottom 10 timestamps
'''q2 = """SELECT *
    FROM ratings
    ORDER BY timestamp
    LIMIT 10;"""
'''
ratings_summary = pysqdf(q)
#print(pysqdf(q2))

In [11]:
ratings_summary.to_csv("ratings_summary.csv")

In [13]:
# Checking movies for outliers
# Check the top 10 timestamps
q = """SELECT *
    FROM movies
    order by movieId;"""
print(pysqdf(q))


       movieId                               title  \
0            1                    Toy Story (1995)   
1            2                      Jumanji (1995)   
2            3             Grumpier Old Men (1995)   
3            4            Waiting to Exhale (1995)   
4            5  Father of the Bride Part II (1995)   
...        ...                                 ...   
62418   209157                           We (2018)   
62419   209159           Window of the Soul (2001)   
62420   209163                    Bad Poems (2018)   
62421   209169                 A Girl Thing (2001)   
62422   209171      Women of Devil's Island (1962)   

                                            genres  
0      Adventure|Animation|Children|Comedy|Fantasy  
1                       Adventure|Children|Fantasy  
2                                   Comedy|Romance  
3                             Comedy|Drama|Romance  
4                                           Comedy  
...                              

In [14]:
# Checking links for outliers
# Check the top 10 timestamps
q = """SELECT *
    FROM links
    order by movieId;"""

print(pysqdf(q))


       movieId   imdbId    tmdbId
0            1   114709     862.0
1            2   113497    8844.0
2            3   113228   15602.0
3            4   114885   31357.0
4            5   113041   11862.0
...        ...      ...       ...
62418   209157  6671244  499546.0
62419   209159   297986   63407.0
62420   209163  6755366  553036.0
62421   209169   249603  162892.0
62422   209171    55323   79513.0

[62423 rows x 3 columns]


#### Checking the min and max of all tables to see if there are any outliers

- No noticable outliers in genomeTags for tag
- No noticable outliers in genomeTags for relevace
- No noticable outliers in tags for timestamps
- No noticable outliers in the movies table
- No noticable outliers in the links table
- Ratings table: User 72314 has 32202 ratings for different movies, the next highest is user 80973 who has 9178 reviews and many of the higher users are closer to that number. There are also User 75308 has 5525 reviews that are all 5 stars, every other user that has all 5 star reviews are around 100 and the majority are in the 20s of reviews. There are some users that have several thousand reviews and have an average rating of less than 1 star, it could be that they are just picky so these users might not be outliers since there are more than just a few that follow this trend. 



